In [1]:
import pandas as pd
import numpy as np
import os
import math
import sys
import glob
# sys.path.append('..')
from tools import haversine, CSVReader_GPS, TimeTranslator, WebCodeTranslator, FillMissingGPSData

## hyper-parameters

In [2]:

## 設定要輸入的gps data folder路徑
gps_file_list = glob.glob(os.path.join('./test_omnieyes/GPS_RawData/', '*.csv'))
gps_file_list.sort()
# gps_file_list.pop(1)
## 輸出檔案說明：
'''
'./test_omnieyes/DF_CSV/' : 資料夾中的csv檔案儲存 map matching完的結果，行車路線經過的Node。

'test_omnieyes/WebVisualizeCode/' : 資料夾中的txt檔案分別為儲存 原始數據/ 行車路線經過的Node 的VisualCode
將內容複製貼到WebSample.html中即可在瀏覽器瀏覽結果。

'''


"\n'./test_omnieyes/DF_CSV/' : 資料夾中的csv檔案儲存 map matching完的結果，行車路線經過的Node。\n\n'test_omnieyes/WebVisualizeCode/' : 資料夾中的txt檔案分別為儲存 原始數據/ 行車路線經過的Node 的VisualCode\n將內容複製貼到WebSample.html中即可在瀏覽器瀏覽結果。\n\n"

## Functions

In [3]:
def GenGPX(df_RawData, save_name):
    ## 1. generate GPX file, 
    lat = df_RawData['lat'].values
    lon = df_RawData['lon'].values
    with open('./GPXdata/template.gpx', 'r')as f:
        txt = f.readlines()
    Nd_trackpoint = []
    with open(save_name, 'w') as f:
        f.writelines(txt)
        for i in range(len(lat)):
            line = '<trkpt lat=\"'+ str(lat[i]) +'\" lon=\"' + str(lon[i]) + '\"><time></time></trkpt>\n'
            Nd_trackpoint.append([lat[i], lon[i]])
            f.write(line)
        f.write('</trkseg>\n</trk>\n</gpx>\n')
        print(save_name + ' saved...')
    return Nd_trackpoint
def ReadGPX(filename, getWebCode=False):
    '''
    INPUT 
        filename: type(String), './filepath/filename.gpx'
        getWebCode: type(Bool), if u want to show results on the website, print the JS code
    OUTPUT
        Nd_trajectory: type([[lat, lon], ...])
        visualweb_code: code       
    '''
    visualweb_code = ''
    Nd_trajectory = []
    if '.res.gpx' in filename:
        lastone_count_ornot = 4
    else:
        lastone_count_ornot = 3
    with open(filename, 'r') as f:
        txt = f.readlines()    
        for i in range(3, len(txt)-lastone_count_ornot):
            word = txt[i].split('"')
            lat = word[1]
            lon = word[3]
            Nd_trajectory.append([float(lat), float(lon)])
            if getWebCode:
                visualweb_code += 'L.marker(['+ lat +', '+ lon + '], {icon: goldIcon}).addTo(mymap).bindPopup("''").openPopup()\n'        
    return Nd_trajectory, visualweb_code


def GraphHopper(gpxfile):
    !bash ./mm.sh {gpxfile}

def SearchNodeID(df_Node, pos):
    '''
    INPUT
        df_Node: Nodes info, type(pd.dataframe)
        pos: type([[lat, lon], ...])
    OUTPUT
        nodeID: type([int, ...]), or None
    '''
    IDList = []
    print('input position: ', pos)
    for pid in range(len(pos)):
        NodeID = None
        lat = pos[pid][0]
        lon = pos[pid][1]
        for i in range(len(df_Node)):
            if math.fabs(df_Node.iloc[i].lat-pos[pid][0]) < 0.00001:
                if math.fabs(df_Node.iloc[i].lon-pos[pid][1]) < 0.00001:
                    NodeID = df_Node.iloc[i].node
                    lat = df_Node.iloc[i].lat
                    lon = df_Node.iloc[i].lon
                    break
        IDList.append([NodeID, lat, lon])
#     print('IDList: ', IDList)
    return IDList
    
def OrthogonalProjection(AB, AP):
    ## projection
    PROJ = (np.dot(AB, AP) / np.dot(AB, AB)) * AB
    return PROJ

def VectorsAngle(A, B):
    ## /pi *180
    return math.acos(np.dot(A, B) / np.sqrt(np.dot(A, A)*np.dot(B, B)))
    
    
def MapMatching(gpxfile, tempfile):
    pass
    data, _ = ReadGPX(gpxfile)
    data_colnames_RawData = ['lat', 'lon']
    NofSeg = 300000
    NofCover = 30
    Nd_trajectory = []
    iters = math.ceil(max((len(data)-NofSeg), 0)/(NofSeg-NofCover)) + 1
#     print(iters)
    ## split gpx data
    for i in range(iters):
        if i != iters-1:
            raw_data = pd.DataFrame(data[(NofSeg-NofCover)*i:(NofSeg-NofCover)*i+NofSeg], columns=data_colnames_RawData)
            GenGPX(raw_data, tempfile)
            print('input points: {}~{}'.format((NofSeg-NofCover)*i, (NofSeg-NofCover)*i+NofSeg))
        else:
            raw_data = pd.DataFrame(data[(NofSeg-NofCover)*i:], columns=data_colnames_RawData)
            GenGPX(raw_data, tempfile)
            print('input points: {}~{}'.format((NofSeg-NofCover)*i, len(data)-1))

        ## Do MapMatching through GraphHopper
        GraphHopper(tempfile)
        
        ## Read Mapping results
        nodes, _ = ReadGPX(tempfile+'.res.gpx')
#         print(Nd_trajectory)
        if len(Nd_trajectory) != 0:
#             if Nd_trajectory[-1] == nodes[0]:
#                 Nd_trajectory.extend(nodes[1:])
#             else:
#                 Nd_trajectory.extend(nodes)
            for nd in nodes:
#                 print(nd)
#                 input()
                if nd in Nd_trajectory:
                    pass
                else:
                    Nd_trajectory.extend([nd])
        else:
            Nd_trajectory.extend(nodes)
            
    return Nd_trajectory

def CSVReader_GPS(data, video_list):           
    raw_data = [] 
    for video_name in video_list:
        d = data.loc[data['video_name']==video_name]
        label = np.array(d['time'])
        longitude = np.array(d['longitude'])
        latitude = np.array(d['latitude'])
        for i in range(len(label)):
            raw_data.append([label[i], latitude[i], longitude[i]])
    return raw_data



## Main

In [6]:

for gps_csv in gps_file_list:
#     print('gps_csv: ', gps_csv)


    tempfile = './GPXdata/temp.gpx'
    all_raw_data = pd.read_csv(gps_csv, encoding = "ISO-8859-1")
    # with open(gps_csv, 'r', encoding = "utf-8")as f:
    #     txt = f.readlines()

    # print(txt)
    index = all_raw_data.keys()
    if 'RITI' in gps_csv:
        n = 2
    else:
        n = 3
    # print(index)
    # print(all_raw_data[index[n]].values)

    raw_lat = all_raw_data[index[n]].values
    raw_lon = all_raw_data[index[n+1]].values
    # print(raw_lon)
    lat_list = []
    lon_list = []
    data_list = []
    for i in range(len(raw_lat)):
        if isinstance(raw_lat[i], float) or isinstance(raw_lat[i], int):
            if math.isnan(raw_lat[i]) or math.isnan(raw_lon[i]):
                continue
            data_list.append([float(raw_lat[i]), float(raw_lon[i])])
                
        elif raw_lat[i].replace('.', '', 1).isdigit():
            data_list.append([float(raw_lat[i]), float(raw_lon[i])])
                
    
    ## RawData Visualization
    txt = ''
    txt_name = os.path.basename(gps_csv)[:-4]+'_RawData.txt'
    for i, nd in enumerate(data_list):
        if n == 2:
            txt += 'L.marker(['+str(nd[0])+', '+str(nd[1]) +'], {icon: blueIcon}).addTo(mymap).bindPopup("'+str(i)+'").openPopup()';
        else:
            txt += 'L.marker(['+str(nd[0])+', '+str(nd[1]) +'], {icon: redIcon}).addTo(mymap).bindPopup("'+str(i)+'").openPopup()';
        txt += ';\n'
    with open(os.path.join('./test_omnieyes/WebVisualizeCode/', txt_name), 'w') as f:
        f.write(txt)
    print(os.path.join('./test_omnieyes/WebVisualizeCode/', txt_name), ' saved...')


    # print(len(data_list))

    ## convert GPS data to Pandas:DataFrame Format
    data_colnames_RawData = ['lat', 'lon']
    df_RawData = pd.DataFrame(data_list, columns=data_colnames_RawData)

    # print(df_RawData[df_RawData.timestamp == '02:03:06'])
#     print('len(df_RawData): ', len(df_RawData))
    
    
    gpx_save_name = os.path.join('./GPXdata/', os.path.basename(gps_csv)[:-4]+'.gpx')
    # Nd_trackpoint = GenGPX(df_RawData, save_name=gpx_save_name)

    ## Gen NewData 
    Nd_trackpoint = GenGPX(df_RawData, save_name=gpx_save_name)

    ## cal. slope
    # slope_start = (Nd_trackpoint[1][1]-Nd_trackpoint[0][1]) / (Nd_trackpoint[1][0]-Nd_trackpoint[0][0])
    # slope_end = (Nd_trackpoint[-1][1]-Nd_trackpoint[-2][1]) / (Nd_trackpoint[-1][0]-Nd_trackpoint[-2][0])
    # print('slope_start, slope_end: ', slope_start, slope_end)
    Nd_trajectory = MapMatching(gpx_save_name, tempfile)      
    # print('Nd_trajectory: ', Nd_trajectory)
    data_colnames_trajectory = ['lat', 'lon']
    Df_trajectory = pd.DataFrame(Nd_trajectory, columns=data_colnames_trajectory)
    print(Df_trajectory.to_json(orient='records'))
    Df_trajectory.to_csv(os.path.join('./test_omnieyes/DF_CSV/', os.path.basename(gps_csv)[:-4]+'_Df_trajectory.csv'), index=0)

    print()
    ## determine distance
    total_dist = 0
    for i in range(len(Nd_trajectory)-1):
        dist = haversine(Nd_trajectory[i][0], Nd_trajectory[i][1], Nd_trajectory[i+1][0], Nd_trajectory[i+1][1])
        total_dist += dist
    print('gps_csv name: ', gps_csv)
    print('number of RawData points: ', len(data_list))
    print('number of map matching nodes: ', len(Nd_trajectory))
    print('total_dist: {} km'.format(total_dist/1000))
    
    
    txt = ''
    txt_name = os.path.basename(gps_csv)[:-4]+'_trajectoryNode.txt'
    for i, nd in enumerate(Nd_trajectory):
        if n == 2:
            txt += 'L.marker(['+str(nd[0])+', '+str(nd[1]) +'], {icon: blueIcon}).addTo(mymap).bindPopup("'+str(i)+'").openPopup()';
        else:
            txt += 'L.marker(['+str(nd[0])+', '+str(nd[1]) +'], {icon: blueIcon}).addTo(mymap).bindPopup("'+str(i)+'").openPopup()';
        txt += ';\n'
    with open(os.path.join('./test_omnieyes/WebVisualizeCode/', txt_name), 'w') as f:
        f.write(txt)
    print(os.path.join('./test_omnieyes/WebVisualizeCode/', txt_name), ' saved...')
    print('-------------------------------------')


./test_omnieyes/WebVisualizeCode/行車紀錄_2020-06-15_5T-336_RawData.txt  saved...
./GPXdata/行車紀錄_2020-06-15_5T-336.gpx saved...
./GPXdata/temp.gpx saved...
input points: 0~2203
Doing ./GPXdata/temp.gpx
loading graph from cache
./GPXdata/temp.gpx
	matches:	414, gps entries:2204
	gpx length:	22143.629 vs 21941.414
	export results to:/home/ubuntu/Desktop/map-matching/./GPXdata/temp.gpx.res.gpx
gps import took:0.47273844s, match took: 1.4989077
[{"lat":25.040921,"lon":121.460921},{"lat":25.041017,"lon":121.461218},{"lat":25.041077,"lon":121.46128},{"lat":25.041261,"lon":121.46141},{"lat":25.041297,"lon":121.461467},{"lat":25.041261,"lon":121.46141},{"lat":25.041077,"lon":121.46128},{"lat":25.041017,"lon":121.461218},{"lat":25.04093,"lon":121.461338},{"lat":25.040831,"lon":121.461432},{"lat":25.039831,"lon":121.46183},{"lat":25.039229,"lon":121.460855},{"lat":25.039178,"lon":121.460651},{"lat":25.039182,"lon":121.460587},{"lat":25.039225,"lon":121.460498},{"lat":25.03973,"lon":121.460303},{"lat


gps_csv name:  ./test_omnieyes/GPS_RawData/行車紀錄_2020-06-15_5T-336.csv
number of RawData points:  2204
number of map matching nodes:  418
total_dist: 21.879705509555993 km
./test_omnieyes/WebVisualizeCode/行車紀錄_2020-06-15_5T-336_trajectoryNode.txt  saved...
-------------------------------------


## References

1. graphhopper installation tutorial [https://github.com/graphhopper/map-matching]

In [5]:
haversine(25.063700,121.657045, 25.060677,121.646430)

1120.7982645861

In [6]:
<trkpt lat="25.061691" lon="121.646451"><time></time></trkpt>

<trkpt lat="25.061691" lon="121.646451"><time></time></trkpt>

remove 61~105

SyntaxError: invalid syntax (<ipython-input-6-78372b235cb8>, line 1)